# Analisar as finanças de sua planilha de dados

Esse programa é uma adaptação que inicialmente foi baseado em um artigo da plataforma Towards DataScience que utiliza principalmente bibliotecas de processamento de dados (Pandas e Numpy) e bibliotecas gráficas (Plotly e Jupyter-Dash) para desenvolver os gráficos baseado em seus dados. ([source link](https://towardsdatascience.com/manage-your-money-with-python-707579202203))

## Estrutura dos dados

Os dados são lidos de uma planilha que conterá as colunas: 
- Carteira
> É para catalogar a origem da transação financeira. (**Exemplos**: *Nubank (Crédito), Nubank (Débito), Nubank (PIX), Cateira física, etc.*)
- Data
> A data que ocorreu a transação financeira.
- Nome
> Identificação ou descrição do que é essa transação financeira.
- Valor
> O valor da transação financeira, tal que é positiva para remunerações e negativa para gastos.
- Categoria
> O tipo de gasto realizado que pode ser, por exemplo, categorizado em **ganhos** (Remuneração, Trabalho, Ajuda Familiar, etc) ou **gastos** (Mercado, Alimentação não-essencial, Bares/Restaurantes, Lazer, Transporte, Compras, Internet / Telefone / Assinaturas, Presentes / Doações, Despesas esporádicas, Urgências / Serviços, Cuidados pessoais, Saúde, Educação, Moradia, etc).

# Observações
1. As categorias precisam ser pré-definidas na estruturação dos dados.



In [ ]:
#@title Vamos inicializar o ambiente

# Instalar o Jupyter-Dash no Colab Notebook
!pip install jupyter-dash

# Importar as bibliotecas de processamento de dados
import pandas as pd                       #to perform data manipulation and analysis
import numpy as np                        #to cleanse data
from datetime import datetime             #to manipulate dates

# Importar as bibliotecas gráficas
import plotly.express as px               #to create interactive charts
import plotly.graph_objects as go         #to create interactive charts
from jupyter_dash import JupyterDash      #to build Dash apps from Jupyter environments
from dash import dcc        #to get components for interactive user interfaces
from dash import html       #to compose the dash layout using Python structures

# Inicializar a leitura do Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#@title Criar banco de dados e Categorias

def create_clean_data():
    dados = pd.DataFrame()
    dados["Nome"] = []
    dados["Valor"] = []
    dados["Data"] = []
    dados["Categoria"] = []
    dados["Carteira"] = []
    return dados

dados = create_clean_data()

# Vamos definir as categorias:

ExpanseCategories = ["Não reconhecido", "Empréstimo", "Mercado", "Alimentação não-essencial", "Bares/Restaurantes", "Lazer", "Transporte", "Compras", "Internet / Telefone / Assinaturas", "Presentes / Doações", "Despesas esporádicas", "Urgências / Serviços", "Cuidados pessoais", "Saúde", "Educação", "Moradia"]
IncomeCategories = ["Remuneração", "Reembolso", "Ajuda Familiar"]

In [ ]:
#@title Vamos abrir a planilha excel no Google Drive (Pasta Financero/Finanças.xlsx)

dados = pd.read_excel("/content/drive/MyDrive/Financeiro/Finanças.xlsx")
display(dados)

In [ ]:
import calendar

df = dados

df['year_month'] = [ f"{calendar.month_name[month]} {year}" for month, year in zip(pd.DatetimeIndex(df['Data']).month, pd.DatetimeIndex(df['Data']).year) ]

display(df)

Expenses = df[df["Categoria"].isin(ExpanseCategories)]
Expenses.Valor = Expenses.Valor*(-1) 

Incomes = df[df["Categoria"].isin(IncomeCategories)]

In [6]:
months = ["January 2021", "February 2021", "March 2021", "April 2021", "May 2021", "June 2021", 
          "July 2021", "August 2021", "September 2021", "October 2021", "November 2021", "December 2021", 
          "January 2022", "February 2022", "March 2022", "April 2022", "May 2022", "June 2022", 
          "July 2022", "August 2022", "September 2022", "October 2022", "November 2022", "December 2022"]

Para fins de ordenamento, será necessário catalogar manualmente a ordem dos meses no formato anterior.

### TODO:
- [ ] Criar categorização das datas automaticamente

In [ ]:
#@title Patrimônio Líquido Pelo Tempo

Net_Worth_Table = df.groupby('year_month')['Valor'].sum().reset_index(name ='sum')
Net_Worth_Table['cumulative sum'] = Net_Worth_Table['sum'].cumsum()

# Vamos ordenar os meses
Net_Worth_Table['year_month'] = pd.Categorical(Net_Worth_Table['year_month'], categories=months, ordered=True)
Net_Worth_Table.sort_values(by=["year_month"], inplace=True)

Net_Worth_Chart = go.Figure(
    data = go.Scatter(x = Net_Worth_Table["year_month"], y = Net_Worth_Table["cumulative sum"]),
    layout = go.Layout(
        title = go.layout.Title(text = "Patrimônio Líquido Pelo Tempo")
    )
)
Net_Worth_Chart.update_layout(
    xaxis_title = "Período",
    yaxis_title = "Patrimônio Líquido (R$)",
    hovermode = 'x unified'
    )
Net_Worth_Chart.update_xaxes(tickangle = 45)
Net_Worth_Chart.show()

# Gastos

In [ ]:
#@title Gastos mensais (Total)

Total_Monthly_Expenses_Table = Expenses.groupby('year_month')['Valor'].sum().reset_index(name = 'sum')

# Vamos ordenar os meses
Total_Monthly_Expenses_Table['year_month'] = pd.Categorical(Total_Monthly_Expenses_Table['year_month'], categories=months, ordered=True)
Total_Monthly_Expenses_Table.sort_values(by=["year_month"], inplace=True)

Total_Monthly_Expenses_Chart = px.bar(Total_Monthly_Expenses_Table, x = "year_month", y = "sum", title = "Gastos mensais total")
Total_Monthly_Expenses_Chart.update_yaxes(title = 'Gastos (R$)', visible = True, showticklabels = True)
Total_Monthly_Expenses_Chart.update_xaxes(title = 'Período', visible = True, showticklabels = True)
Total_Monthly_Expenses_Chart.show()

In [ ]:
#@title Gastos mensais (Por categoria)

Expenses_Breakdown_Table = pd.pivot_table(Expenses, values = ['Valor'], index = ['year_month', "Categoria"], aggfunc=sum).reset_index()

def sort_table(table):
    table['year_month'] = pd.Categorical(table['year_month'], categories=months, ordered=True)
    table.sort_values(by=["year_month"], inplace=True)
    table = table.reset_index(drop=True)
    return table

def populate_and_sort_table(table):
    categorias = np.unique(table["Categoria"])
    months_years = np.unique(table["year_month"])


    for categoria in categorias:
        for date in months_years:
            if not ((table['year_month'] == date) & (table['Categoria'] == categoria)).any():
                table = table.append({"year_month": date, "Categoria": categoria, "Valor": 0.0}, ignore_index=True)

    return sort_table(table)

Expenses_Breakdown_Table = populate_and_sort_table(Expenses_Breakdown_Table)

Expenses_Breakdown_Chart = px.line(Expenses_Breakdown_Table, x='year_month', y="Valor", title="Gastos por Categoria", color = 'Categoria')

Expenses_Breakdown_Chart.update_yaxes(title='Gastos (R$)', visible=True, showticklabels=True)
Expenses_Breakdown_Chart.update_xaxes(title='Período', visible=True, showticklabels=True)

Expenses_Breakdown_Chart.update_layout(xaxis=dict(type='category'))

Expenses_Breakdown_Chart.update_traces(mode='markers+lines')

Expenses_Breakdown_Chart.show()

# Rendas

In [ ]:
#@title Rendas mensais (Total)

Total_Monthly_Income_Table = Incomes.groupby('year_month')['Valor'].sum().reset_index(name = 'sum')
Total_Monthly_Income_Table = sort_table(Total_Monthly_Income_Table)
Total_Monthly_Income_Chart = px.bar(Total_Monthly_Income_Table, x = "year_month", y = "sum", title = "Rendas mensais total")
Total_Monthly_Income_Chart.update_yaxes(title = 'Renda (R$)', visible = True, showticklabels = True)
Total_Monthly_Income_Chart.update_xaxes(title = 'Período', visible = True, showticklabels = True)
Total_Monthly_Income_Chart.show()

In [ ]:
#@title Rendas mensais (Por categoria)

Incomes_Breakdown_Table = pd.pivot_table(Incomes, values = ['Valor'], index = ['Categoria', 'year_month'], aggfunc=sum).reset_index()
Incomes_Breakdown_Table = populate_and_sort_table(Incomes_Breakdown_Table)
Incomes_Breakdown_Chart = px.line(Incomes_Breakdown_Table, x='year_month', y="Valor", title="Renda por categoria", color = 'Categoria')
Incomes_Breakdown_Chart.update_yaxes(title='Renda (R$)', visible=True, showticklabels=True)
Incomes_Breakdown_Chart.update_xaxes(title='Período', visible=True, showticklabels=True)
Incomes_Breakdown_Chart.show()

# Deploy app

In [ ]:
# Build App
app = JupyterDash(__name__)
latest_date = "October"
app.layout = html.Div([   
    html.Div([
        html.H1(str(latest_date)+" Personal Finance Summary",style={'text-align':'center'}),
        dcc.Graph(figure = Net_Worth_Chart)
    ]),  
    html.Div([
        dcc.Graph(figure = Total_Monthly_Expenses_Chart)
    ]),   
    html.Div([
        dcc.Graph(figure = Expenses_Breakdown_Chart)
    ]),
    html.Div([
        dcc.Graph(figure = Total_Monthly_Income_Chart)
    ]),   
    html.Div([
        dcc.Graph(figure = Incomes_Breakdown_Chart)
    ])
])
    
# Run app and display result
app.run_server(mode='external')

#Click on the link below to access the "Personal Finances Summary"

# Extra: Higinização, Organização e Formatação dos dados

In [ ]:
#@title Funções adicionais para ajudar a organizar e formatar os dados

# Importar extrato do guiabolso!
def import_extrato_guiabolso(file, dados):
    extrato = pd.read_csv(file)

    for i in range(0, len(extrato)):
        dados = dados.append({
            "Nome": extrato["Transação"][i],
            "Valor": extrato["Valor da transação"][i],
            "Data": extrato["Data da transação"][i],
            "Categoria": extrato["Categoria"][i],
            "Carteira": extrato["Conta"][i]
        }, ignore_index=True)
    
    # EXEMPLOS DE HIGIENIZAÇÂO DOS DADOS
    # Vamos limpar as transações internas
    dados = dados[dados.Nome != "Pagamento recebido"]

    # Vamos renomear categorias
    dados.Categoria.replace(["Contas residenciais"], "Moradia",inplace=True)
    dados.Categoria.replace(["Bares / Restaurantes"], "Bares/Restaurantes",inplace=True)
    dados.Categoria.replace(["TV / Internet / Telefone"], "Internet / Telefone / Assinaturas",inplace=True)
    dados.Categoria.replace(["Transferência"], "Não reconhecido",inplace=True)

    dados["Categoria"] = np.where(dados["Nome"].str.contains('PESSOA A'), 'Lazer', dados["Categoria"])
    dados["Categoria"] = np.where(dados["Nome"].str.contains('ESTABELECIMENTO B'), 'Bares/Restaurantes', dados["Categoria"])
    dados["Categoria"] = np.where(dados["Nome"].str.contains('Uber|PAYMEE|Clickbus|Auttar'), 'Transporte', dados["Categoria"])

    return dados

# Importar extrato do Nubank
def import_extrato_nubank(file, dados):
    extrato = pd.read_csv(file)

    for i in range(0, len(extrato)):
        dados = dados.append({
            "Nome": extrato["Descrição"][i],
            "Valor": extrato["Valor"][i],
            "Data": extrato["Data"][i],
            "Categoria": "Não reconhecido",
            "Carteira": "Nubank",
        }, ignore_index=True)
    
    # EXEMPLOS DE HIGIENIZAÇÂO DOS DADOS
    # Vamos limpar as transações internas
    dados = dados[dados.Nome != "Pagamento recebido"]

    # Vamos renomear categorias
    dados.Categoria.replace(["Contas residenciais"], "Moradia",inplace=True)
    dados.Categoria.replace(["Bares / Restaurantes"], "Bares/Restaurantes",inplace=True)
    dados.Categoria.replace(["TV / Internet / Telefone"], "Internet / Telefone / Assinaturas",inplace=True)
    dados.Categoria.replace(["Transferência"], "Não reconhecido",inplace=True)

    dados["Categoria"] = np.where(dados["Nome"].str.contains('PESSOA A'), 'Lazer', dados["Categoria"])
    dados["Categoria"] = np.where(dados["Nome"].str.contains('ESTABELECIMENTO B'), 'Bares/Restaurantes', dados["Categoria"])
    dados["Categoria"] = np.where(dados["Nome"].str.contains('Uber|PAYMEE|Clickbus|Auttar'), 'Transporte', dados["Categoria"])

    return dados

In [ ]:
#@title Exemplo de organização dos dados

# Vamos ler os dados de Janeiro e Fevereiro de 2022
dados = create_clean_data() # Iniciamos o DataFrame limpo
dados = import_extrato_guiabolso("/content/extrato_guiabolso 2022.csv", dados) # Importamos a planilha csv
dados.sort_values(by=["Data"], inplace=True) # Ordenamos a planilha csv por data
dados = dados.reset_index(drop=True)[0:64] # Escolhemos os primeiro 65 elementos (nessa planilha são as transações de Jan/Fev)

# Vamos importar transações que ocorrem mensalmente mas não estão catalogadas
for i in range(0, 9): #Mar, Abril, Maio, Junho, Julho, Agosto, Setembro, Outubro, Novembro
    dados = dados.append({
        "Nome": "Aluguel",
        "Valor": -750,
        "Data": f"2022-{i+3}-15",
        "Categoria": "Moradia",
        "Carteira": "Nubank (Débito)"
    }, ignore_index=True)
    dados = dados.append({
        "Nome": "Recebimento de trabalho",
        "Valor": 2000,
        "Data": f"2022-{i+3}-15",
        "Categoria": "Remuneração",
        "Carteira": "Nubank"
    }, ignore_index=True)

# Vamos ler os dados de 2021
dados = import_extrato_guiabolso("/content/extrato_guiabolso 2021.csv", dados)

# Vamos concatenar e ordenar os dados concatena
dados['Data']= pd.to_datetime(dados['Data']) # Vamos garantir que todas as datas estejam no mesmo tipo
dados.sort_values(by=["Data"], inplace=True) # Vamos ordenar as transações por datas
dados = dados.reset_index(drop=True) # Vamos resetar a contagem do index

display(dados) # Vamos visualizar os dados